# Agent and Environment
We should first come up with action way to represent states and actions in the game. Each action can be simply mapped to moving either four directions. We also represent the current state of the agent represent the immediate cells around it.

In [99]:
# Define the state space and the action space
state_space = 3**8 # The number of possible states, each cell being Wall, Empty, or Dot, Agent
action_space = 4 # The number of possible actions

# Q-table
Now we initialize the Q table being state_space * action_space which initially is filled with zeroes.

In [100]:
import numpy as np

# Initialize the Q-table with zeros
Q = np.zeros((state_space, action_space))

def encode(state, isShortened=False):
    """
    Encodes either an environmen or a state to a base-3 integer

    Args:
        state (lits(string)**2): either the environment or state
        isShortened (bool, optional): Becomes true if encoding state. Defaults to False.

    Returns:
        int: returns the encoding of the current state either from the environment or the state
    """
    result = 0
    # if the input matrix is the whole environment
    if isShortened == False:
        for row in range(len(state)):
            for column in range(len(state[row])):
                if state[row][column] == 'A':
                    # Translate the 3*3 grid to a 8-cell-long line, ignoring the agent's location
                    temp = [state[row-1][i] for i in range(column-1, column+2)]
                    temp.append(state[row][column-1])
                    temp.append(state[row][column+1])
                    temp += [state[row+1][i] for i in range(column-1, column+2)]
                    #print(temp)
                    # Encode in base 3
                    for cell in temp:
                        result *= 3
                        if (cell == 'W'):
                            result += 0
                        elif (cell == 'D'):
                            result += 1
                        elif (cell == 'E'):
                            result += 2
    # if the input matrix is just a state
    else:
        # Encode in base 3
        for row in range(len(state)):
            for column in range(len(state[row])):
                # ignore the agent's location to save space
                if row == 1 and column == 1:
                    continue
                result *= 3
                if (state[row][column] == 'W'):
                    result += 0
                elif (state[row][column] == 'D'):
                    result += 1
                elif (state[row][column] == 'E'):
                    result += 2
    return result

def decode(encoding):
    """
    returns the decoded int representing a state

    Args:
        encoding (int): an incoding of the current state

    Returns:
        list(string)**2: a 3*3 matrix showing the state
    """
    # Define a 3*3 matrix with place-holder A's
    state = [['A' for _ in range(3)]for _ in range(3)]
    for row in range(2, -1, -1):
        for column in range(2, -1, -1):
            # In this case it is the agent's cell, which we have pre-filled with A
            if(row == 1 and column == 1):
                continue
            
            # Decode in base 3
            remainder = encoding % 3
            encoding = encoding // 3
            if remainder == 0:
                state[row][column] = 'W'
            elif remainder == 1:
                state[row][column] = 'D'
            elif remainder == 2:
                state[row][column] = 'E'

    return state

# Parameters
Now before training the mode we should define our parameters.

In [101]:
# Define the parameters, NOTE: naming is based on a Stanford course I had seen online before
learning_rate = 0.65 # How much to update the Q-value (A.K.A. Alpha)
discount_factor = 0.5 # How much to discount the future reward (A.K.A. Gamma)
max_episodes = 10000 # How many episodes to train the agent
max_iteration = 1000 # How many iterations to go each episode
epsilon = 0.1 # How much to explore randomly

In [102]:
import random

def get_possible_reward(state, action):
    """
    gets a state and a planned action and 
    calculates the possible IMMEDIATE reward for that action

    Args:
        state (int_encoding): and encoding of the current state of the agent
        action (int): a string showing what the next action of agent is going to be

    Returns:
        int: the possible reward to get
    """
    # Initial reward
    reward = 0
    # Get the destination cell of next action
    temp = decode(state)
    if action == 'up':
        temp = temp[0][1]
    elif action == 'down':
        temp = temp[2][1]
    elif action == 'right':
        temp = temp[1][2]
    elif action == 'left':
        temp = temp[1][0]
    
    # Calculate the reward. NOTE: cell cannot be WALL.
    if temp == 'D':
        reward += 10
    elif temp == 'E':
        reward -= 2

    return reward

def move_next_state(environment, action):
    """
    gets an environment and proceeds to run the action on the environment

    Args:
        environment (list(string) ** 2): a list of lists of characters showing the current environment
        action (string): a string showin the next action to take

    Returns:
        list(string) ** 2: the new environment after taking the action
    """
    for row in range(1, len(environment)):
        for column in range(1, len(environment[row])):
            if environment[row][column] == 'A':
                #print (row, column)
                # Clear the previous cell of agent
                environment[row][column] = 'E'
                # Move the agent to the next position
                if action == 'up':
                    environment[row-1][column] = 'A'
                elif action == 'down':
                    environment[row+1][column] = 'A'
                elif action == 'right':
                    environment[row][column+1] = 'A'
                elif action == 'left':
                    environment[row][column-1] = 'A'
                    
                #print(*environment, sep='\n')
                #print("in move_next_state, action=", action)
                #print("\n\n\n")
                
                return environment

def get_next_action(state):
    """
    gets the environment and current state, based on the epsilon value either selects a random action
    or uses the Q table to look for the most profitable action to take

    Args:
        environment (list(string)**2): the environment of the game
        state (int_encoding): the encoding of the current state of the agent

    Returns:
        string: (random/best) action to take in the current state
    """
    temp = decode(state)
    explore = random.random()
    # RANDOM ACTION
    if explore > epsilon:

        # Choose one the four directions that is not blocked by a wall (W) randomly
        range = 0
        if temp[0][1] != 'W':
            range += 1
        if temp[1][0] != 'W':
            range += 1
        if temp[1][2] != 'W':
            range += 1
        if temp[2][1] != 'W':
            range += 1
        move = random.randint(0, (range-1))
        
        # Find the chosen direction
        iterator = 0
        if temp[0][1] != 'W':
            if move == iterator:
                return 'up'
            iterator += 1
        if temp[1][2] != 'W':
            if move == iterator:
                return 'right'
            iterator += 1
        if temp[2][1] != 'W':
            if move == iterator:
                return 'down'

        #print("DEBUG: TOOK LEFT AS THE ONLY REMAINING OPTION", iterator, range, temp)            # DEBUG
        return 'left'
    
    # BEST ACTION
    else:
        # Next action first assumed to be unknown
        best_move = 'NULL'
        best_move_score = -1e9
        # Find the action with maximum possible reward
        temp2 = Q[state][0]#up
        if (temp[0][1] != 'W' and temp2 > best_move_score):
            best_move_score = temp2
            best_move = 'up'
        temp2 = Q[state][1]#down
        if (temp[2][1] != 'W' and temp2 > best_move_score):
            best_move_score = temp2
            best_move = 'down'
        temp2 = Q[state][2]#left
        if (temp[1][0] != 'W' and temp2 > best_move_score):
            best_move_score = temp2
            best_move = 'left'
        temp2 = Q[state][3]#right
        if (temp[1][2] != 'W' and temp2 > best_move_score):
            best_move_score = temp2
            best_move = 'right'

        # if non found something is not right! the Q-Table values have gone way down.
        if best_move == 'NULL':
            raise Exception("THE AGENT HAS GOTTEN STUCK BETWEEN 4 WALLS", temp)                     # DEBUG

        return best_move
    
def update_Q(prev_state, prev_action, reward, new_state):
    """
    Updating the Q values after taking an action

    Args:
        prev_state (int_encoding): an encoding of the previous state agent was in
        prev_action (string): a string showin the previous action agent has taken
        reward (int): an integer showing the immediate reward gathered by taking the previous action
        newState (int_encoding): an encoding of the current state agent is in
    """

    # Encode action to integer
    if prev_action == 'up':
        prev_action = 0
    elif prev_action == 'down':
        prev_action = 1
    elif prev_action == 'left':
        prev_action = 2
    elif prev_action == 'right':
        prev_action = 3

    # Update Q using the rule taught in class
    max_Q = Q[new_state][3]
    for i in range(3):
        max_Q = max(Q[new_state][i], max_Q)
    Q[prev_state][prev_action] = (((1-learning_rate) * Q[prev_state][prev_action]) + 
                                (learning_rate * (reward + (discount_factor * max_Q))))
    return

In [103]:
for episode in range(max_episodes):
    # Reseting the environment
    environment = [['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
               ['W', 'A', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W'],
               ['W', 'D', 'W', 'W', 'W', 'D', 'W', 'W', 'W', 'D', 'W'],
               ['W', 'D', 'W', 'D', 'D', 'D', 'D', 'D', 'W', 'D', 'W'],
               ['W', 'D', 'D', 'D', 'W', 'E', 'W', 'D', 'D', 'D', 'W'],
               ['W', 'D', 'W', 'D', 'W', 'E', 'W', 'D', 'W', 'D', 'W'],
               ['W', 'D', 'W', 'D', 'D', 'W', 'D', 'D', 'W', 'D', 'W'],
               ['W', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W'],
               ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W']]

    # Reseting the epsilon value
    global epsilon
    epsilon = 0.1

    # Counting number of D's
    dot_count = 0
    for list in environment:
        dot_count += list.count('D')

    # Counting number of steps the agent takes
    stepcount = 0
    while (dot_count > 0 and stepcount < max_iteration):
        stepcount += 1
        dot_count = 0
        
        # Counting number of D's
        for list in environment:
            dot_count += list.count('D')

        # If in the very last episode put the chance of taking randomized actions to be zero
        if episode >= 9999:
            epsilon = 1.1
            # For Step By Steo Visual representation
            """print("IN MAIN:")
            print(*environment, sep="\n")
            print(*decode(encode(environment)), sep="\n")"""
        # Encode the current state
        current_state = encode(environment)
        # Choose action
        action = get_next_action(current_state)
        # Get Reward
        reward = get_possible_reward(current_state, action)
        # Update Environment
        environment = move_next_state(environment, action)
        # Update Q
        update_Q(current_state, action, reward, encode(environment))
        # Update epsilon
        epsilon += 0.001
        # Cap epsilon at 0.9, we don't want the randomness to end unless in the very last episode
        if epsilon > 0.9:
            epsilon = 0.9
    
    # Print the number of steps taken in the very last episode
    if (episode >= 9999):
        print(stepcount)


73


In [121]:
# PART 5 OF THE HOMEWORK
for i in range (state_space):
    if (sum(Q[i]) != 0 ):    
        print(i-1, Q[i])

29 [ 0.         19.98647338  0.         19.89897327]
32 [ 0.          1.97782671  0.         19.98306769]
41 [ 0.          7.96902384  0.         14.03956076]
50 [ 0.          1.49992146  0.         11.85131713]
56 [ 0.         19.99316651  0.          2.56431622]
59 [ 0.         -1.99985815  0.         -1.99926823]
65 [ 0.         19.99979254  0.          1.9991656 ]
68 [ 0.          7.99668439  0.         -1.900198  ]
74 [ 0.         19.75049476  0.         -0.46118012]
77 [ 0.         -1.99921162  0.         -1.0474756 ]
86 [ 0.          1.33180602 16.65650491  0.        ]
87 [ 0.          7.975045   19.02669526  0.        ]
88 [ 0.          1.80981266 10.98911933  0.        ]
113 [ 0.          1.99238336 17.1058326  17.79272864]
134 [ 0.          0.         10.86652357 -1.99999684]
135 [ 0.          0.         18.40874627  7.12398368]
136 [ 0.          0.         14.82965139 -1.79543719]
137 [ 0.         13.8979545  14.93985511  1.99465478]
140 [ 0.          1.82836993 15.26573719 